In [1]:
# Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, box
import fiona
import pandas as pd
import geopandas as gpd
import rasterio as rio
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Source data, SRTM & GLIHT
srtm_source = os.path.join('/Users/arbailey/natcap/idb/data/source/srtm/nasa','SRTM30m_Andros.vrt')
glihtdtm_source = os.path.join('/Users/arbailey/natcap/idb/data/source/gliht/Andros_May2015/lidar/geotiff/Andros_May2015_DTM','Andros_May2015_DTM.vrt')
glihtchm_source = os.path.join('/Users/arbailey/natcap/idb/data/source/gliht/Andros_May2015/lidar/geotiff/Andros_May2015_CHM','Andros_May2015_CHM.vrt' )
# Working directory
work_dir = '/Users/arbailey/natcap/idb/data/work/andros_mangrovehgt'
os.chdir(work_dir)

In [3]:
# GLiHT Point layer with SRTM and Max Height attributes already assigned
gliht_srtm_hmax95_lyr = 'glihtchm_srtm_hmax95'

# Directories and data of mangrove layers to be sampled
mangroves_dir = '/Users/arbailey/natcap/idb/data/work/mangroves'
mangroves_gpkg = os.path.join(mangroves_dir, 'mangroves_andros.gpkg')
## Layers
# # Simard - Mangrove Height
# mangrovehgt_tif = 'Mangrove_hmax95_andros.tif'
# mangrovehgt_source = os.path.join(mangroves_dir, mangrovehgt_tif)
# Giri - Global Mangrove Forest
gmf_lyr = 'gmf_andros'
# Spalding - World Atlas of Mangroves
wam_lyr = 'wam2010_andros_droppedfields'
# Bunting - Global Mangrove Watch
gmw19962016_lyr = 'gmw_1996_2016_andros'
gmw2015_lyr = 'GMW_2015_v2_andros'
# TNC Landsat Mangroves
tnclandsat_lyr = 'mar_mangroves_2006'
# NatCap / TNC mangroves minus coppice and DBLE
ncmangwet_lyr = 'and_mar_wetlandmangrove_NATCAP' #'and_mar_wetlandmangrove_NATCAP_sp'

# Output Geopackage
out_gpkg = os.path.join(work_dir, 'gliht_srtm.gpkg')

In [4]:
%%time
# Load mangrove polygon layers in GeoDataFrames
gmf = gpd.read_file(mangroves_gpkg, layer=gmf_lyr) # Giri, Global Mangrove Forests
wam = gpd.read_file(mangroves_gpkg, layer=wam_lyr) # Spalding, World Atlas of Mangroves
gmw19962016 = gpd.read_file(mangroves_gpkg, layer=gmw19962016_lyr)  # Bunting, Global Mangrove Watch - all years
gmw2015 = gpd.read_file(mangroves_gpkg, layer=gmw2015_lyr) # Bunting, Global Mangrove Watch 2015
tnclandsat = gpd.read_file(mangroves_gpkg, layer=tnclandsat_lyr) # TNC Landsat (2002) Mangrove Layer
ncmangwet = gpd.read_file(mangroves_gpkg, layer=ncmangwet_lyr) # TNC/NatCap Mangrove/wetlands layer 

CPU times: user 24 s, sys: 1.22 s, total: 25.3 s
Wall time: 25.6 s


In [5]:
# GLobal Mangrove Watch 2015
# Modify/Update Attributes
print(gmw2015.head)
gmw2015_att = 'gmw2015'
gmw2015 = gmw2015.rename(columns={'pxlval': gmw2015_att})
gmw2015 = gmw2015[[gmw2015_att,'geometry']]  # include only gmw2015 and geometry columns
gmw2015[gmw2015_att] = pd.to_numeric(gmw2015[gmw2015_att], downcast='integer')
gmw2015

<bound method NDFrame.head of        ogc_fid  pxlval  Id                                           geometry
0      15724.0     1.0   0  POLYGON ((-77.74444 24.00000, -77.74422 24.000...
1      15725.0     1.0   0  POLYGON ((-77.74400 24.00000, -77.74378 24.000...
2      15726.0     1.0   0  POLYGON ((-77.75267 23.99978, -77.75244 23.999...
3      15727.0     1.0   0  POLYGON ((-77.74711 24.00000, -77.74578 24.000...
4      15728.0     1.0   0  POLYGON ((-77.74511 23.99956, -77.74467 23.999...
...        ...     ...  ..                                                ...
9704  552271.0     1.0   0  POLYGON ((-77.81733 24.00156, -77.81711 24.001...
9705  552272.0     1.0   0  POLYGON ((-77.74756 24.00089, -77.74689 24.000...
9706  552273.0     1.0   0  POLYGON ((-77.76933 24.00089, -77.76911 24.000...
9707  552274.0     1.0   0  POLYGON ((-77.74422 24.00044, -77.74400 24.000...
9708  552275.0     1.0   0  POLYGON ((-77.74622 24.00089, -77.74600 24.000...

[9709 rows x 4 columns]>


,gmw2015,geometry
0,1,"POLYGON ((-77.74444 24.00000, -77.74422 24.000..."
1,1,"POLYGON ((-77.74400 24.00000, -77.74378 24.000..."
2,1,"POLYGON ((-77.75267 23.99978, -77.75244 23.999..."
3,1,"POLYGON ((-77.74711 24.00000, -77.74578 24.000..."
4,1,"POLYGON ((-77.74511 23.99956, -77.74467 23.999..."
...,...,...
9704,1,"POLYGON ((-77.81733 24.00156, -77.81711 24.001..."
9705,1,"POLYGON ((-77.74756 24.00089, -77.74689 24.000..."
9706,1,"POLYGON ((-77.76933 24.00089, -77.76911 24.000..."
9707,1,"POLYGON ((-77.74422 24.00044, -77.74400 24.000..."


In [6]:
%%time
# TNC Landsat Mangroves
# Modify/Update Attributes
print(tnclandsat.head)
tnclandsat_att = 'tnclandsat'
tnclandsat[tnclandsat_att] = np.where(tnclandsat['grid_code']>=19, 1, 2)  # dense = 1, sparse = 2
tnclandsat = tnclandsat[[tnclandsat_att,'geometry']] # include only ncmangwet and geometry columns
tnclandsat

<bound method NDFrame.head of         area  perimeter  and_ply_bl  and_ply__1  grid_code            target  \
0      0.009       0.66         579         578         20  Sparse Mangroves   
1      0.015       0.84         602         601         20  Sparse Mangroves   
2      0.009       0.54         658         657         20  Sparse Mangroves   
3      0.008       0.54        1126        1125         20  Sparse Mangroves   
4      0.006       0.42        1131        1130         20  Sparse Mangroves   
...      ...        ...         ...         ...        ...               ...   
26454  0.013       0.66      101916      101915         20  Sparse Mangroves   
26455  0.008       0.54      102001      102000         20  Sparse Mangroves   
26456  0.009       0.60      102013      102012         20  Sparse Mangroves   
26457  0.004       0.36      102064      102063         20  Sparse Mangroves   
26458  0.006       0.36      102087      102086         20  Sparse Mangroves   

       ac

,tnclandsat,geometry
0,1,"MULTIPOLYGON (((181242.000 2808898.750, 181272..."
1,1,"MULTIPOLYGON (((181452.000 2808568.750, 181422..."
2,1,"MULTIPOLYGON (((181542.000 2808268.750, 181422..."
3,1,"MULTIPOLYGON (((183912.000 2805538.750, 183942..."
4,1,"MULTIPOLYGON (((184092.000 2805388.750, 184062..."
...,...,...
26454,1,"MULTIPOLYGON (((216612.000 2607658.750, 216612..."
26455,1,"MULTIPOLYGON (((216852.000 2607328.750, 216822..."
26456,1,"MULTIPOLYGON (((217092.000 2607298.750, 217092..."
26457,1,"MULTIPOLYGON (((216852.000 2607088.750, 216822..."


In [7]:
%%time
# NatCap/TNC Mangroves & Wetlands layer
# Modify/Update Attributes
print(ncmangwet.columns)
print(ncmangwet.head)
ncmangwet_att = 'ncmangwet'
ncmangwet[ncmangwet_att] = 1 # all polygons in the layer are mangroves, so set ncmangwet = 1
ncmangwet = ncmangwet[[ncmangwet_att,'geometry']] # include only ncmangwet and geometry columns
ncmangwet
## Keep getting errors with this layer.  Maybe something wrong with the data source ?

Index(['Id', 'geometry'], dtype='object')
<bound method NDFrame.head of    Id                                           geometry
0   0  POLYGON ((186615.748 2740172.278, 186615.748 2...
1   0  POLYGON ((209840.748 2739952.278, 209840.748 2...
2   0  POLYGON ((176025.748 2791307.278, 176025.748 2...
3   0  POLYGON ((188130.748 2783822.278, 188130.748 2...
4   0  POLYGON ((218712.000 2612488.750, 218832.000 2...
5   0  POLYGON ((229615.748 2668272.278, 229610.748 2...
6   0  POLYGON ((226565.748 2674922.278, 226565.748 2...
7   0  POLYGON ((225345.748 2702757.278, 225342.000 2...
8   0  POLYGON ((182800.748 2760297.278, 182790.748 2...>
CPU times: user 21.8 s, sys: 780 ms, total: 22.5 s
Wall time: 23.5 s


,ncmangwet,geometry
0,1,"POLYGON ((186615.748 2740172.278, 186615.748 2..."
1,1,"POLYGON ((209840.748 2739952.278, 209840.748 2..."
2,1,"POLYGON ((176025.748 2791307.278, 176025.748 2..."
3,1,"POLYGON ((188130.748 2783822.278, 188130.748 2..."
4,1,"POLYGON ((218712.000 2612488.750, 218832.000 2..."
5,1,"POLYGON ((229615.748 2668272.278, 229610.748 2..."
6,1,"POLYGON ((226565.748 2674922.278, 226565.748 2..."
7,1,"POLYGON ((225345.748 2702757.278, 225342.000 2..."
8,1,"POLYGON ((182800.748 2760297.278, 182790.748 2..."


In [8]:
# Import the previously joined point file layer into GeoDataFrame (includes wam and gmf)
%time gliht_srtm_hmax95_mangroves1 = gpd.read_file(out_gpkg, layer='glihtchm_srtm_hmax95_mangroves1')
gliht_srtm_hmax95_mangroves1

CPU times: user 27min 45s, sys: 42min 43s, total: 1h 10min 29s
Wall time: 1h 46min 26s


,z_m,x_utm17n,y_utm17n,srtm_m,srtm_id,hmax95,hmax95_id,gliht_ptid,gmf,wam,geometry
0,0.0,788785.5,2787858.5,0,21282071,NaN,14210037,1,NaN,NaN,POINT (-78.13478 25.17915)
1,0.0,788781.5,2787857.5,0,21282071,NaN,14210036,2,NaN,NaN,POINT (-78.13482 25.17914)
2,0.0,788784.5,2787857.5,0,21282071,NaN,14210037,3,NaN,NaN,POINT (-78.13479 25.17914)
3,0.0,788776.5,2787856.5,0,21282070,NaN,14210036,4,NaN,NaN,POINT (-78.13487 25.17913)
4,0.0,788778.5,2787856.5,0,21282071,NaN,14210036,5,NaN,NaN,POINT (-78.13485 25.17913)
...,...,...,...,...,...,...,...,...,...,...,...
58146866,0.0,802646.5,2691223.5,0,43944037,NaN,44267460,58146867,NaN,NaN,POINT (-78.01828 24.30487)
58146867,0.0,802641.5,2691222.5,0,43951238,NaN,44267460,58146868,NaN,NaN,POINT (-78.01833 24.30486)
58146868,0.0,802644.5,2691221.5,0,43951238,NaN,44267460,58146869,NaN,NaN,POINT (-78.01830 24.30485)
58146869,0.0,802646.5,2691220.5,0,43951238,NaN,44267460,58146870,NaN,NaN,POINT (-78.01828 24.30484)


In [9]:
# Reproject TNC Landsat Data to same CRS as source point data (EPSG 4326)
%time tnclandsat_4326 = tnclandsat.to_crs(gliht_srtm_hmax95_mangroves1.crs)
tnclandsat_4326

CPU times: user 8.75 s, sys: 977 ms, total: 9.73 s
Wall time: 12.9 s


,tnclandsat,geometry
0,1,"MULTIPOLYGON (((-78.16718 25.36281, -78.16688 ..."
1,1,"MULTIPOLYGON (((-78.16502 25.35988, -78.16532 ..."
2,1,"MULTIPOLYGON (((-78.16406 25.35720, -78.16525 ..."
3,1,"MULTIPOLYGON (((-78.13990 25.33308, -78.13960 ..."
4,1,"MULTIPOLYGON (((-78.13808 25.33177, -78.13838 ..."
...,...,...
26454,1,"MULTIPOLYGON (((-77.77599 23.55458, -77.77597 ..."
26455,1,"MULTIPOLYGON (((-77.77358 23.55165, -77.77387 ..."
26456,1,"MULTIPOLYGON (((-77.77122 23.55142, -77.77122 ..."
26457,1,"MULTIPOLYGON (((-77.77353 23.54948, -77.77382 ..."


In [12]:
# Reproject NatCap/TNC mangroves/wetlands layer to same CRS as source point data (EPSG 4326)
%time ncmangwet_4326 = ncmangwet.to_crs(gliht_srtm_hmax95_mangroves1.crs)
ncmangwet_4326

CPU times: user 3min 36s, sys: 1min 31s, total: 5min 8s
Wall time: 7min 9s


,ncmangwet,geometry
0,1,"POLYGON ((-78.09822 24.74420, -78.09822 24.744..."
1,1,"POLYGON ((-77.86880 24.74678, -77.86880 24.746..."
2,1,"POLYGON ((-78.21476 25.20308, -78.21476 25.203..."
3,1,"POLYGON ((-78.09308 25.13814, -78.09308 25.138..."
4,1,"POLYGON ((-77.75635 23.59852, -77.75517 23.598..."
5,1,"POLYGON ((-77.65985 24.10370, -77.65990 24.103..."
6,1,"POLYGON ((-77.69109 24.16317, -77.69108 24.163..."
7,1,"POLYGON ((-77.70841 24.41409, -77.70844 24.414..."
8,1,"POLYGON ((-78.14047 24.92489, -78.14057 24.924..."


In [13]:
%%time
# Join Attributes of GMW 2015 polygons to master point layer
join3 = gpd.sjoin(gliht_srtm_hmax95_mangroves1, gmw2015, how="left")
join3 = join3.drop(columns='index_right')
join3

CPU times: user 57min 41s, sys: 43min 58s, total: 1h 41min 39s
Wall time: 2h 17min 3s


,z_m,x_utm17n,y_utm17n,srtm_m,srtm_id,hmax95,hmax95_id,gliht_ptid,gmf,wam,geometry,gmw2015
0,0.0,788785.5,2787858.5,0,21282071,NaN,14210037,1,NaN,NaN,POINT (-78.13478 25.17915),NaN
1,0.0,788781.5,2787857.5,0,21282071,NaN,14210036,2,NaN,NaN,POINT (-78.13482 25.17914),NaN
2,0.0,788784.5,2787857.5,0,21282071,NaN,14210037,3,NaN,NaN,POINT (-78.13479 25.17914),NaN
3,0.0,788776.5,2787856.5,0,21282070,NaN,14210036,4,NaN,NaN,POINT (-78.13487 25.17913),NaN
4,0.0,788778.5,2787856.5,0,21282071,NaN,14210036,5,NaN,NaN,POINT (-78.13485 25.17913),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
58146866,0.0,802646.5,2691223.5,0,43944037,NaN,44267460,58146867,NaN,NaN,POINT (-78.01828 24.30487),NaN
58146867,0.0,802641.5,2691222.5,0,43951238,NaN,44267460,58146868,NaN,NaN,POINT (-78.01833 24.30486),NaN
58146868,0.0,802644.5,2691221.5,0,43951238,NaN,44267460,58146869,NaN,NaN,POINT (-78.01830 24.30485),NaN
58146869,0.0,802646.5,2691220.5,0,43951238,NaN,44267460,58146870,NaN,NaN,POINT (-78.01828 24.30484),NaN


In [14]:
# export to Geopackage
%time join3.to_file("gliht_srtm.gpkg", layer='glihtchm_srtm_hmax95_mangroves2', driver="GPKG")

CPU times: user 2h 58s, sys: 33min 5s, total: 2h 34min 3s
Wall time: 3h 24min 31s


In [15]:
%%time
# Join attributes of TNC Landsat polygons to master point layer
join4 = gpd.sjoin(join3, tnclandsat_4326, how="left")
join4 = join4.drop(columns='index_right')
join4

CPU times: user 1h 31min 23s, sys: 45min 1s, total: 2h 16min 24s
Wall time: 2h 55min 25s


,z_m,x_utm17n,y_utm17n,srtm_m,srtm_id,hmax95,hmax95_id,gliht_ptid,gmf,wam,geometry,gmw2015,tnclandsat
0,0.0,788785.5,2787858.5,0,21282071,NaN,14210037,1,NaN,NaN,POINT (-78.13478 25.17915),NaN,NaN
1,0.0,788781.5,2787857.5,0,21282071,NaN,14210036,2,NaN,NaN,POINT (-78.13482 25.17914),NaN,NaN
2,0.0,788784.5,2787857.5,0,21282071,NaN,14210037,3,NaN,NaN,POINT (-78.13479 25.17914),NaN,NaN
3,0.0,788776.5,2787856.5,0,21282070,NaN,14210036,4,NaN,NaN,POINT (-78.13487 25.17913),NaN,NaN
4,0.0,788778.5,2787856.5,0,21282071,NaN,14210036,5,NaN,NaN,POINT (-78.13485 25.17913),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58146866,0.0,802646.5,2691223.5,0,43944037,NaN,44267460,58146867,NaN,NaN,POINT (-78.01828 24.30487),NaN,NaN
58146867,0.0,802641.5,2691222.5,0,43951238,NaN,44267460,58146868,NaN,NaN,POINT (-78.01833 24.30486),NaN,NaN
58146868,0.0,802644.5,2691221.5,0,43951238,NaN,44267460,58146869,NaN,NaN,POINT (-78.01830 24.30485),NaN,NaN
58146869,0.0,802646.5,2691220.5,0,43951238,NaN,44267460,58146870,NaN,NaN,POINT (-78.01828 24.30484),NaN,NaN


In [16]:
# export to Geopackage
%time join4.to_file("gliht_srtm.gpkg", layer='glihtchm_srtm_hmax95_mangroves3', driver="GPKG")

CPU times: user 2h 9min 46s, sys: 32min 47s, total: 2h 42min 34s
Wall time: 3h 30min 26s


In [17]:
%%time
# Join attributes of NatCap/TNC mangrove/wetland polygons to master point layer
join5 = gpd.sjoin(join4, ncmangwet_4326, how="left")
join5 = join5.drop(columns='index_right')
join5

Exception ignored in: <function Handle.__del__ at 0x7f801930cbf8>
Traceback (most recent call last):
  File "/Users/arbailey/miniconda3/envs/geoenv/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/Users/arbailey/miniconda3/envs/geoenv/lib/python3.7/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'


AttributeError: 'NoneType' object has no attribute 'intersection'

In [18]:
# export to Geopackage
%time join5.to_file("gliht_srtm.gpkg", layer='glihtchm_srtm_hmax95_mangroves4', driver="GPKG")

NameError: name 'join5' is not defined